# Service-Layer API
<sup>Authored by Annapoorani G (annag@cisco.com)</sup>

Bring your protocol or controller on IOS-XR to interact with the network infrastructure layer components using Cisco IOS-XR Service-Layer API. 
Service-Layer API is a model-driven API over Google-defined remote procedure call (gRPC)
gRPC that enables you to bring your applications, routing protocols, controllers in a rich set of languages including C++, Python, GO, and many more.
In IOS-XR, routing protocols use Routing Information Base (RIB), the MPLS label manager, BFD, and other modules, to program the forwarding plane and these are exposed through the service layer API. 
The Service-Layer API gives direct access to the Network Infrastructure Layer (Service-Adaptation Layer). Therefore, you have the following advantages:

* High Performance: Direct access to the Network Infrastructure Layer without going through a Network state database results in higher performance than equivalent Management APIs. For example, Batch updates straight to the RIB, Label Switch Database (over gRPC).
* Flexibility: Gives you the flexibility to bring your Protocol or Controller over gRPC
* Offload low-level tasks to IOS -XR: IOS-XR infrastructure layer handles conflict resolution, transactional notifications, scalability, and data plane abstraction, hence, you can focus on higher-layer protocols and controller logic


## Components of Service-Layer API

![SLAPI](SL-API.png)

## Before You Begin

Before you begin to enable Service Layer, ensure to complete the following prerequisites.
* [Bring up your emulator](#step1)
* [Enable Service Layer on a router](#step2)
* [Install gRPC and regenerate bindings](#step3)
* [Run pre-packaged tutorials](#step4)

## <a name="step1"></a>Bring up your Emulator


>In order to set up the emulator to build our network, you have to execute the cell below by clicking the play button at the top of this notebook. 

>In the cell below, the python module **servicelayer** sets up the python enviroment and the simulated 2-router test bed along with the base configurations of IPv4 addresses, MPLS configurations, Loopback interfaces, etc. After the import, the below code block initialises and brings up the emulator using **sim.init** and **sim.up** API calls.


In [4]:
from servicelayer import *
sim = Vxr()
sim.no_image_copy=True
sim.clean()
print("Sim clean: Done")
print("Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.")

try:
    sim.start(cfg)
    status = sim.status()
    print("Sim status: ", status)
except Exception as err:
    print("Sim launch failed (%s)" % str(err))

INFO:pyvxr.vxr:v1.1.0 2021-04-27 10:58 output_dir:vxr.out
INFO:pyvxr.vxr:bd8e02377cc7:/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer
INFO:pyvxr.vxr:Extracting vxr version from '/opt/cisco/vxr2/latest/setup.sh' file.
INFO:pyvxr.vxr_session:Starting a local bash session for user:vxr


Sim clean: Done
Simulation starting. Please wait for the Sim status message. This may take 3-10 minutes.


INFO:pyvxr.sim:Launch: sim_dir:/nobackup/vxr/pyvxr/10kxy79358 sim_rel:/opt/cisco/vxr2/latest
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)
INFO:pyvxr.sim:Starting vxr: 'sim --skiphomecheck -n '
INFO:pyvxr.sim:Vxr up on host localhost
INFO:pyvxr.vxr:Getting port vector files for:r1, r2
INFO:pyvxr.console:r2:wait for XR login prompt (console output captured in vxr.out/logs/console.r2.log)
INFO:pyvxr.console:r1:wait for XR login prompt (console output captured in vxr.out/logs/console.r1.log)
INFO:pyvxr.console:r1:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:r2:entering new XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:r1:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.console:r2:entering XR username 'cisco', password 'cisco123'
INFO:pyvxr.bringup:r1:login successful
INFO:pyvxr.bringup:r2:login successful
INFO:pyvxr.bringup:r1:wait for IOS XR RUN state
INFO:pyvxr.bringup:r2:wa

Sim status:  {'localhost': 'running'}


Play the cell below to obtain the telnet and ssh IP-addresses and ports to access the devices.

In [5]:
print('Consoles can be reached by:')
print(get_telnet_cmd(sim, 'r1'), '\n', get_telnet_cmd(sim, 'r2'))
print('or better:')
print(get_ssh_cmd(sim, 'r1'), '\n', get_ssh_cmd(sim, 'r2'))
print('The password is cisco123')

Consoles can be reached by:
telnet 172.17.0.2 38470 
 telnet 172.17.0.2 39588
or better:
ssh cisco@172.17.0.2 -p61559 
 ssh cisco@172.17.0.2 -p60008
The password is cisco123


#### <a name="step11"></a>View the redirect port details


>Note: Make a note of the redirect port (xr_redir57021) and the IP address. Will need it later.

In [6]:
port = sim.ports()
r1_ip = str(port['r1']['HostAgent'])
r1_grpc = str(port['r1']['xr_redir57021'])
print(r1_ip, r1_grpc)

172.17.0.2 63979


## <a name="step2"></a>Enable Service Layer on a Router

### Enable gRPC 


Configure gRPC port to access the gRPC server over the external NAT IP address
> Configure gRPC port on either router to be 57777 in order to access the gRPC server over the external NAT IP address.

In [7]:
import telnetlib
ports = sim.ports()
loginp1 = telnetlib.Telnet(str(ports['r1']['HostAgent']) , str(ports['r1']['serial0']))
loginp2 = telnetlib.Telnet(str(ports['r2']['HostAgent']) , str(ports['r2']['serial0']))

In [8]:
loginp1.write(('''
show route
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show route
Tue Apr 27 11:02:03.527 UTC

Codes: C - connected, S - static, R - RIP, B - BGP, (>) - Diversion path
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2, E - EGP
       i - ISIS, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, su - IS-IS summary null, * - candidate default
       U - per-user static route, o - ODR, L - local, G  - DAGR, l - LISP
       A - access/subscriber, a - Application route
       M - mobile route, r - RPL, t - Traffic Engineering, (!) - FRR Backup path

Gateway of last resort is not set

C    192.168.254.0/24 is directly connected, 00:01:07, MgmtEth0/RP0/CPU0/0
L    192.168.254.99/32 is directly connected, 00:01:07, MgmtEth0/RP0/CPU0/0
C    198.51.100.0/24 is directly connected, 00:00:23, FourHundredGigE0/0/0/0
L    198.51.100.1/3

In [9]:
loginp1.write(('''
configure
grpc
port 57021
commit
root
exit
show  grpc status
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#configure
Tue Apr 27 11:02:35.775 UTC
RP/0/RP0/CPU0:P1(config)#grpc
RP/0/RP0/CPU0:P1(config-grpc)#port 57021
RP/0/RP0/CPU0:P1(config-grpc)#commit
Tue Apr 27 11:02:36.362 UTC
RP/0/RP0/CPU0:P1(config-grpc)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#show  grpc status
Tue Apr 27 11:02:37.466 UTC



### Enable Service-Layer

Service-Layer APIs are served up over gRPC. So simply enable service-layer under the gRPC config mode:

In [10]:
loginp1.write(('''
config
grpc
service-layer
commit
root
exit
show service-layer state
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****
*************************show gRPC status**********************
---------------------------------------------------------------
transport                       :     grpc
access-family                   :     tcp4
TLS                             :     enabled
trustpoint                      :     
listening-port                  :     57021
max-request-per-user            :     10
max-request-total               :     128
max-streams                     :     32
max-streams-per-user            :     32
vrf-socket-ns-path              :     global-vrf
_______________________________________________________________
*************************End of showing status*****************
RP/0/RP0/CPU0:P1#
RP/0/RP0/CPU0:P1#config
Tue Apr 27 11:02:39.254 UTC
RP/0/RP0/CPU0:P1(config)#grpc
RP/0/RP0/CPU0:P1(config-grpc)#service-layer
RP/0/RP0/CPU0:P1(config-grpc-sl)#commit
Tue Apr 27 11:02:39.653 UTC
RP/0/RP0/CPU0:P1(config-grpc-sl)#root
RP/0/RP0/CPU0:P1(config)#exi

### Disable TLS under grpc

 Configure no-tls under the gRPC config mode. In the show grpc status output, TLS should get set to disabled.

In [11]:
loginp1.write(('''
configure
grpc
no-tls
commit
root
exit
show grpc status
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#configure
Tue Apr 27 11:02:42.202 UTC
RP/0/RP0/CPU0:P1(config)#grpc
RP/0/RP0/CPU0:P1(config-grpc)#no-tls
RP/0/RP0/CPU0:P1(config-grpc)#commit
Tue Apr 27 11:02:42.550 UTC
RP/0/RP0/CPU0:P1(config-grpc)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#show grpc status
Tue Apr 27 11:02:42.847 UTC
*************************show gRPC status**********************
---------------------------------------------------------------
transport                       :     grpc
access-family                   :     tcp4
TLS                             :     disabled
trustpoint                      :     NotSet
listening-port                  :     57021
max-request-per-user            :     10
max-request-total               :     128
max-streams                     :     32
max-streams-per-user            :     32
vrf-socket-ns-path              :     global-vrf
_______________________________________________________________
********************

### Verify final gRPC configuration

At the end, your final gRPC configuration on Router P1 should look something like:

In [12]:
loginp1.write(('''
show running-config grpc
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show running-config grpc
Tue Apr 27 11:02:45.006 UTC
grpc
 port 57021
 no-tls
 service-layer
 !
!

RP/0/RP0/CPU0:P1#


In [13]:
loginp1.write(('''
show service-layer state
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show service-layer state
Tue Apr 27 11:02:47.011 UTC
------------service layer state----------
config on:                   YES
standby connected :          NO
idt done:                    NO
blocked on ndt:              NO
connected to RIB for IPv4:   YES
connected to RIB for IPv6:   YES
Initialization state:        service ready
pending requests:            0         
BFD Connection:              UP
MPLS Connection:             UP
Interface Connection:        UP
Objects accepted:            NO
interface registered:        NO
bfd registered for IPv4:     NO
bfd registered for IPv6:     NO
RP/0/RP0/CPU0:P1#


## Check that the gRPC port is operational

The gRPC server runs as a Linux process on the IOS-XR system. It therefore opens up a port in the kernel on the Router Processor (RP) module.
This port also gets programmed into the LPTS (Local Packet Transport Services) component of IOS-XR that maintains a distributed table across all nodes (RP, Linecards) of the Router to enable clients to connect to the gRPC port over distributed line card ports along with the Management Port.


## gRPC port in LPTS

In [14]:
loginp1.write(('''
show lpts  pifib brief  | i 57021
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show lpts  pifib brief  | i 57021
Tue Apr 27 11:02:50.357 UTC
 IPv4       default  TCP    any          0/RP0/CPU0                   any,57021 any
RP/0/RP0/CPU0:P1#


## gRPC port in the RP kernel

In [15]:
loginp1.write(('''
bash netstat -anput | grep 57021
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#bash netstat -anput | grep 57021
Tue Apr 27 11:02:52.365 UTC
tcp        0      0 0.0.0.0:57021           0.0.0.0:*               LISTEN      10547/emsd      

RP/0/RP0/CPU0:P1#


## <a name="step3"></a>Install gRPC and regenerate bindings

The SL API client connects to the router over gRPC. So, the steps we intend to perform as part of this section are:

* Install the protoc compiler python-pip provides protoc using a grpc-tools package.

* Provide the model (.proto) files to the compiler and generate bindings  
>This generated code is then used as a set of libraries to create our own client code.

>> First, install git and clone the Service-Layer Object Model Repository. 

**Set proxy settings if behind a firewall**

In [16]:
# Proxy settings - Update as per your settings
!export http_proxy=http://proxy.esl.cisco.com:80/
!export https_proxy=http://proxy.esl.cisco.com:80/
!export ftp_proxy=http://proxy.esl.cisco.com:80/
!export no_proxy=.cisco.com
!export HTTP_PROXY=http://proxy.esl.cisco.com:80/
!export HTTPS_PROXY=http://proxy.esl.cisco.com:80/
!export FTP_PROXY=http://proxy.esl.cisco.com:80/

In [17]:
# Install git
!sudo apt-get -y install git
# Clone the service layer API code from Github
!git clone https://github.com/Cisco-Service-Layer/service-layer-objmodel.git -b v0.0.1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  git-man less
Suggested packages:
  git-daemon-run | git-daemon-sysvinit git-doc git-el git-email git-gui gitk
  gitweb git-cvs git-mediawiki git-svn
The following NEW packages will be installed:
  git git-man less
0 upgraded, 3 newly installed, 0 to remove and 1 not upgraded.
Need to get 4832 kB of archives.
After this operation, 34.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 less amd64 487-0.1 [112 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 git-man all 1:2.17.1-1ubuntu0.8 [804 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 git amd64 1:2.17.1-1ubuntu0.8 [3916 kB]
Fetched 4832 kB in 1s (3374 kB/s)
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package less.
(Reading database ..

>Install grpc tools

In [18]:
pip install grpcio grpcio-tools --ignore-installed

     |████████████████████████████████| 4.2 MB 26.4 MB/s eta 0:00:01
     |████████████████████████████████| 2.5 MB 42.9 MB/s eta 0:00:01████▉   | 2.3 MB 42.9 MB/s eta 0:00:01
     |████████████████████████████████| 784 kB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 49.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install googleapis-common-protos

     |████████████████████████████████| 198 kB 24.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


>> Generate Python bindings

In [20]:
cd service-layer-objmodel/grpc/python

/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer/service-layer-objmodel/grpc/python


The gen-bindings.sh script first cleans up the existing genpy/ directory where the bindings is  created and then proceeds to loop through the proto files running the protoc utility from the grpc_tools package.


> View the content of the gen-bindings.sh script:

In [21]:
cat gen-bindings.sh

#!/bin/bash
#
# Copyright (c) 2016 by cisco Systems, Inc. 
# All rights reserved.
#
cd ../protos
printf "Generating Python bindings..."
protoc -I ./ --python_out=../python/src/genpy/ --grpc_out=../python/src/genpy/ --plugin=protoc-gen-grpc=`which grpc_python_plugin` *.proto
cd ../python/src/genpy; 2to3 -w *
echo "Done"


>> Run the gen-bindings.sh script. These bindings are generated from the proto files and represent the python libraries that can be imported into your client code to provide the RPCs to be used to interact with the required functionality vertical.

In [22]:
!./gen-bindings.sh

Generating Python bindings..../gen-bindings.sh: line 8: protoc: command not found
RefactoringTool: Skipping optional fixer: buffer
RefactoringTool: Skipping optional fixer: idioms
RefactoringTool: Skipping optional fixer: set_literal
RefactoringTool: Skipping optional fixer: ws_comma
RefactoringTool: No changes to sl_bfd_common_pb2.py
RefactoringTool: No changes to sl_bfd_ipv4_pb2.py
RefactoringTool: No changes to sl_bfd_ipv4_pb2_grpc.py
RefactoringTool: No changes to sl_bfd_ipv6_pb2.py
RefactoringTool: No changes to sl_bfd_ipv6_pb2_grpc.py
RefactoringTool: No changes to sl_common_types_pb2.py
RefactoringTool: No changes to sl_global_pb2.py
RefactoringTool: No changes to sl_global_pb2_grpc.py
RefactoringTool: No changes to sl_interface_pb2.py
RefactoringTool: No changes to sl_interface_pb2_grpc.py
RefactoringTool: No changes to sl_mpls_pb2.py
RefactoringTool: No changes to sl_mpls_pb2_grpc.py
RefactoringTool: No changes to sl_route_common_pb2.py
RefactoringTool: No changes to sl_route_

> Install grpcio and ipaddress packages. The grpcio package is a must to actually create a channel and connect to the router over gRPC.
The ipaddress python package is used to manage ip addresses in python.

In [23]:
%cd src/genpy/
!pip install ipaddress

/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer/service-layer-objmodel/grpc/python/src/genpy


## <a name="step4"></a>Run pre-packaged tutorials

Let's run some of the representative tutorials that are packaged along with the service-layer-objmodel git repo. For more information about the tutorials, see [Running the tutorial](https://github.com/Cisco-Service-Layer/service-layer-objmodel/tree/master/grpc/python/src/tutorial#quick).

All the tutorials, expect the following environment variables to be set before running them:


* SERVER_IP: IP address over which the router's gRPC server is reachable. 

* SERVER_PORT: TCP port over which the router's gRPC server is reachable.

The port amd IP address can be found from the output that you got from  the step print (sim.get_console_ports()), see [View redirect port details](#step11)

>View the availble pre-packaged tutorials

In [24]:
%cd ..
%cd tutorial/
!ls -l

/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer/service-layer-objmodel/grpc/python/src
/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer/service-layer-objmodel/grpc/python/src/tutorial
total 80
-rw-r--r-- 1 vxr vxr 14189 Apr 27 11:03 README.md
-rw-r--r-- 1 vxr vxr    72 Apr 27 11:03 __init__.py
-rw-r--r-- 1 vxr vxr  5823 Apr 27 11:03 client_init.py
-rw-r--r-- 1 vxr vxr 12028 Apr 27 11:03 interface.py
-rw-r--r-- 1 vxr vxr 10329 Apr 27 11:03 mpls_ilm.py
-rw-r--r-- 1 vxr vxr  5435 Apr 27 11:03 quickstart.py
-rw-r--r-- 1 vxr vxr 12857 Apr 27 11:03 route.py
-rw-r--r-- 1 vxr vxr  3617 Apr 27 11:03 vrf.py


In [25]:
# Using "find" and "sed" to convert "GigabitEthernet" to "FourHundredGigE" in the python files because Cisco8000 has only FourHundredGigE ports
!find *.py -type f -exec sed -i 's/GigabitEthernet/FourHundredGigE/g' {} \;

### Basic Client Initialization: client_init.py

This tutorial utilizes the initialization RPC to set up a notification channel with the service-layer gRPC server running on the router. This notification channel is used by the client to be notified of any server error conditions or any disconnect messages. You cannot utilize any of the service-layer functionality verticals if you don't keep a notification channel active.

This is usually done by initiating the notification channel in a separate thread inside your code, so that the rest of code can continue to execute while the notification channel remains active within the thread.

client_init.py will simply connect, set up a channel and disconnect. Its init code will be run inside a separate thread in the next set of tutorials that leverage it.

In [26]:
%set_env https_proxy=
%set_env SERVER_IP=$r1_ip
%set_env SERVER_PORT=$r1_grpc
!echo $SERVER_IP
!echo $SERVER_PORT
!python client_init.py

env: https_proxy=
env: SERVER_IP=172.17.0.2
env: SERVER_PORT=63979
172.17.0.2
63979
Using GRPC Server IP(172.17.0.2) Port(63979)
Global thread spawned
Server Returned 0x501, Version 0.3.0
Successfully Initialized, connection established!
Max VRF Name Len     : 33
Max Iface Name Len   : 64
Max Paths per Entry  : 128
Max Prim per Entry   : 64
Max Bckup per Entry  : 64
Max Labels per Entry : 3
Min Prim Path-id     : 1
Max Prim Path-id     : 64
Min Bckup Path-id    : 65
Max Bckup Path-id    : 128
Max Remote Bckup Addr: 2


### Register/Unregister against the Route Vertical

This tutorial utilizes the client_init.py code to maintain the notification channel for it while it proceeds to register against a particular vrf (vrf default). This accomplishes registration for the Route vertical described earlier.
Only post registration with the Route vertical can the RIB manipulations be carried out.

In [27]:
!python vrf.py

Using GRPC Server IP(172.17.0.2) Port(63979)
Global thread spawned
Server Returned 0x501, Version 0.3.0
Successfully Initialized, connection established!
Max VRF Name Len     : 33
Max Iface Name Len   : 64
Max Paths per Entry  : 128
Max Prim per Entry   : 64
Max Bckup per Entry  : 64
Max Labels per Entry : 3
Min Prim Path-id     : 1
Max Prim Path-id     : 64
Min Bckup Path-id    : 65
Max Bckup Path-id    : 128
Max Remote Bckup Addr: 2
VRF SL_REGOP_REGISTER Success!
VRF SL_REGOP_EOF Success!
VRF SL_REGOP_UNREGISTER Success!


The initial dump is the same as the earlier run of client_init.py since vrf.py utilizes client_init.py. The last set of messages show that the client was able to:

Successfully Register for the Route vertical (for vrf: default).
Send an EOF (used to flush out stale routes - marked by the registration process- should remind folks of the Mark and Sweep technique used in Garbage Collection algorithms. This is utilized whenever controllers/clients intend to resync routes with the IOS-XR RIB in the event of either a client or server failure).
Successfully Unregister for the Route Vertical (used to disconnect and flush out all the routes from the particular vrf).

### Add Routes to IOS-XR RIB: quickstart.py


The quickstart.py tutorial uses vrf.py (which in turn uses client_init.py as mentioned above) to register against the Route vertical. It then utilizes RPCs in the IPv4 Route vertical to program IPv4 routes into the IOS-XR RIB.

In [28]:
!python quickstart.py

Using GRPC Server IP(172.17.0.2) Port(63979)
Global thread spawned
Server Returned 0x502, Version 0.3.0
Successfully Initialized, connection established!
Max VRF Name Len     : 33
Max Iface Name Len   : 64
Max Paths per Entry  : 128
Max Prim per Entry   : 64
Max Bckup per Entry  : 64
Max Labels per Entry : 3
Min Prim Path-id     : 1
Max Prim Path-id     : 64
Min Bckup Path-id    : 65
Max Bckup Path-id    : 128
Max Remote Bckup Addr: 2
VRF SL_REGOP_REGISTER Success!
VRF SL_REGOP_EOF Success!
Route SL_OBJOP_ADD Success!


To verify this, ssh into the router and do a dump of the RIB using show route:

In [29]:
loginp1.write(('''
show route
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show route
Tue Apr 27 11:08:00.602 UTC

Codes: C - connected, S - static, R - RIP, B - BGP, (>) - Diversion path
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2, E - EGP
       i - ISIS, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, su - IS-IS summary null, * - candidate default
       U - per-user static route, o - ODR, L - local, G  - DAGR, l - LISP
       A - access/subscriber, a - Application route
       M - mobile route, r - RPL, t - Traffic Engineering, (!) - FRR Backup path

Gateway of last resort is not set

a    20.0.0.0/24 [2/0] via 10.10.10.1, 00:00:01, FourHundredGigE0/0/0/0
                 [2/0] via 10.10.10.2, 00:00:01, FourHundredGigE0/0/0/0
a    20.0.1.0/24 [2/0] via 10.10.10.1, 00:00:01, FourHundredGigE0/0/0/0
                 [2/0] via 10.1

### Register and Listen for interface state events: interface.py

This tutorial showcases how to register against the interface vertical, set up event streams for a certain set of interfaces and then start thread to receive notifications of interface events as we shut/no-shut registered interfaces.
In this example, you register for the state events of three interfaces, namely: FourHundredGigE0/0/0/0, FourHundredGigE0/0/0/1 and MgmtEth0/RP0/CPU0/0.


In [30]:
!python interface.py

Using GRPC Server IP(172.17.0.2) Port(63979)
Global thread spawned
Server Returned 0x502, Version 0.3.0
Successfully Initialized, connection established!
Max VRF Name Len     : 33
Max Iface Name Len   : 64
Max Paths per Entry  : 128
Max Prim per Entry   : 64
Max Bckup per Entry  : 64
Max Labels per Entry : 3
Min Prim Path-id     : 1
Max Prim Path-id     : 64
Min Bckup Path-id    : 65
Max Bckup Path-id    : 128
Max Remote Bckup Addr: 2
ErrStatus {
}

ErrStatus {
}

StatusSummary {
}

ErrStatus {
}
MaxInterfacesPerBatch: 1024

Eof: true
ErrStatus {
}
Entries {
  SLIfInfo {
    Name: "FourHundredGigE0/0/0/0"
  }
}
Entries {
  SLIfInfo {
    Name: "FourHundredGigE0/0/0/1"
  }
}
Entries {
  SLIfInfo {
    Name: "MgmtEth0/RP0/CPU0/0"
  }
}

Starting listener for interface events
^C
Unregistering...
ErrStatus {
}



You'll notice that the client is still running, because it is listening for interface state events in a persistent thread. 

<b>Important: Select interrupt kernel to proceed.</b>

**1. From the menu, Select Kernel**

**2. Select Interrupt Kernel**

Optional: If you trigger the shut/no-shut of interface FourHundredGigE0/0/1 and see how the client reacts.
Connect to the router P1 using ssh and shut/no-shut of interface FourHundredGigE0/0/1. These events are being streamed to the client.


In [31]:
loginp1.write(('''
configure
interface FourHundredGigE0/0/0/1
shutdown
commit
root
exit
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#configure
Tue Apr 27 11:08:14.227 UTC
RP/0/RP0/CPU0:P1(config)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:P1(config-if)#shutdown
RP/0/RP0/CPU0:P1(config-if)#commit
Tue Apr 27 11:08:14.613 UTC
RP/0/RP0/CPU0:P1(config-if)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#


In [32]:
loginp1.write(('''
configure
interface FourHundredGigE0/0/0/1
no shutdown
commit
root
exit
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#configure
Tue Apr 27 11:08:16.242 UTC
RP/0/RP0/CPU0:P1(config)#interface FourHundredGigE0/0/0/1
RP/0/RP0/CPU0:P1(config-if)#no shutdown
RP/0/RP0/CPU0:P1(config-if)#commit
Tue Apr 27 11:08:16.615 UTC
RP/0/RP0/CPU0:P1(config-if)#root
RP/0/RP0/CPU0:P1(config)#exit
RP/0/RP0/CPU0:P1#


The Received HeartBeat messages show up as the client receives heartbeats periodically from the Server. The event SL_INTERFACE_EVENT_TYPE_INTERFACE_INFO gives us more information on the type of interface event received: 

In [33]:
!python interface.py

Using GRPC Server IP(172.17.0.2) Port(63979)
Global thread spawned
Server Returned 0x502, Version 0.3.0
Successfully Initialized, connection established!
Max VRF Name Len     : 33
Max Iface Name Len   : 64
Max Paths per Entry  : 128
Max Prim per Entry   : 64
Max Bckup per Entry  : 64
Max Labels per Entry : 3
Min Prim Path-id     : 1
Max Prim Path-id     : 64
Min Bckup Path-id    : 65
Max Bckup Path-id    : 128
Max Remote Bckup Addr: 2
ErrStatus {
}

ErrStatus {
}

StatusSummary {
}

ErrStatus {
}
MaxInterfacesPerBatch: 1024

Eof: true
ErrStatus {
}
Entries {
  SLIfInfo {
    Name: "FourHundredGigE0/0/0/0"
  }
  IfState: SL_IF_STATE_UP
  SeqNum: 2
}
Entries {
  SLIfInfo {
    Name: "FourHundredGigE0/0/0/1"
  }
  IfState: SL_IF_STATE_DOWN
  SeqNum: 2
}
Entries {
  SLIfInfo {
    Name: "MgmtEth0/RP0/CPU0/0"
  }
  IfState: SL_IF_STATE_UP
  SeqNum: 1
}

Starting listener for interface events
^C
Unregistering...
ErrStatus {
}



You'll notice that the client is still running, because it is listening for interface state events in a persistent thread. 

<b>Important: Select interrupt kernel to proceed.</b>

**1. From the menu, Select Kernel**

**2. Select Interrupt Kernel**

### Allocate Local labels and Create ILM entries: mpls_ilm.py

This tutorial showcases how to register against the mpls vertical, allocate label blocks and then utilize these labels to create ILM (incoming label map) to forwarding function entries.


In [35]:
!find mpls_ilm.py -type f -exec sed -i 's/0\/0\/0\/1/0\/0\/0\/0/g' {} \;
!python mpls_ilm.py

Using GRPC Server IP(172.17.0.2) Port(63979)
Global thread spawned
Server Returned 0x502, Version 0.3.0
Successfully Initialized, connection established!
Max VRF Name Len     : 33
Max Iface Name Len   : 64
Max Paths per Entry  : 128
Max Prim per Entry   : 64
Max Bckup per Entry  : 64
Max Labels per Entry : 3
Min Prim Path-id     : 1
Max Prim Path-id     : 64
Min Bckup Path-id    : 65
Max Bckup Path-id    : 128
Max Remote Bckup Addr: 2
Error code for mpls SL_OBJOP_ADD is 0x8004
Error code for mpls SL_OBJOP_UPDATE is 0xa


Hop onto router P1 and dump the allocated label blocks. Notice the labels marked Static(A):Service-layer.  These were allocated by the mpls_ilm.py client.

In [36]:
loginp1.write(('''
show mpls ldp nei
show route
show  mpls label table
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show mpls ldp nei
Tue Apr 27 11:10:27.635 UTC

RP/0/RP0/CPU0:P1#show route
Tue Apr 27 11:10:27.692 UTC

Codes: C - connected, S - static, R - RIP, B - BGP, (>) - Diversion path
       D - EIGRP, EX - EIGRP external, O - OSPF, IA - OSPF inter area
       N1 - OSPF NSSA external type 1, N2 - OSPF NSSA external type 2
       E1 - OSPF external type 1, E2 - OSPF external type 2, E - EGP
       i - ISIS, L1 - IS-IS level-1, L2 - IS-IS level-2
       ia - IS-IS inter area, su - IS-IS summary null, * - candidate default
       U - per-user static route, o - ODR, L - local, G  - DAGR, l - LISP
       A - access/subscriber, a - Application route
       M - mobile route, r - RPL, t - Traffic Engineering, (!) - FRR Backup path

Gateway of last resort is not set

a    20.0.0.0/24 [2/0] via 10.10.10.1, 00:02:28, FourHundredGigE0/0/0/0
                 [2/0] via 10.10.10.2, 00:02:28, FourHundredGigE0/0/0/0
a    20.0.1.0/24 [2/0] via 10.10.10.1, 

Now verify the mpls forwarding entries on the router:

In [37]:
loginp1.write(('''
show mpls forwarding
''').encode('ascii'))
line = loginp1.read_until(b'/r/n',2)
print("***** VIEW FROM TELNET CONSOLE *****")
print(line.decode())

***** VIEW FROM TELNET CONSOLE *****

RP/0/RP0/CPU0:P1#show mpls forwarding
Tue Apr 27 11:10:31.480 UTC
Local  Outgoing    Prefix             Outgoing     Next Hop        Bytes       
Label  Label       or ID              Interface                    Switched    
------ ----------- ------------------ ------------ --------------- ------------
24000  Unlabelled  20.0.0.0/24        FH0/0/0/0    10.10.10.1      0           
       Unlabelled  20.0.0.0/24        FH0/0/0/0    10.10.10.2      0           
24001  Unlabelled  20.0.1.0/24        FH0/0/0/0    10.10.10.1      0           
       Unlabelled  20.0.1.0/24        FH0/0/0/0    10.10.10.2      0           
24002  Unlabelled  20.0.2.0/24        FH0/0/0/0    10.10.10.1      0           
       Unlabelled  20.0.2.0/24        FH0/0/0/0    10.10.10.2      0           
24003  Unlabelled  20.0.3.0/24        FH0/0/0/0    10.10.10.1      0           
       Unlabelled  20.0.3.0/24        FH0/0/0/0    10.10.10.2      0           
24004  Unlabelle

 This showcases a SWAP entry created by the client. This entry will receive any packet with incoming-label=34000, swap it with label=10065 and forward it to the nexthop interface FourHundEthernet0/0/0/1 and nexthop address=12.1.1.20.

Of course, you get to play with the usual mpls label operations like : PUSH, POP_AND_FORWARD, POP_AND_LOOKUP_IPV4 and POP_AND_LOOKUP_IPV6 to create ILM entries based on your use case.

In [38]:
# close our telnet
loginp1.close()

In [39]:
cd ../../../../../

/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer


In [40]:
# Delete downloaded git repository
!pwd
!rm -rf service-layer-objmodel
%set_env https_proxy=http://proxy.esl.cisco.com:80/

/home/vxr/notebooks/Put-Technology-to-Work/ServiceLayer
env: https_proxy=http://proxy.esl.cisco.com:80/


In [41]:
# Close the sim
sim.stop()
sim.clean()

INFO:pyvxr.vxr:Stopping sim on host localhost (dir /nobackup/vxr/pyvxr/10kxy79358)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.vxr:Cleaning sim on host localhost (dir /nobackup/vxr/pyvxr/10kxy79358)
INFO:pyvxr.sim:Stopping previous simulation (if any)
INFO:pyvxr.sim:Cleaning previous simulation (if any)


In [42]:
# Clean up sim scratch space
shutil.rmtree(sim_dir)